# Gather data from various source into the analysis repository

In [1]:
# imports
import os
import shutil
import numpy as np
import pandas as pd
import yaml
import git
import tempfile
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))

import pint
unit_registry = pint.UnitRegistry()

from PLBenchmarks import targets, ligands, edges
from IPython.core.display import HTML

from tqdm.notebook import tqdm

import benchmarkpl
path = benchmarkpl.__path__[0]

_ColormakerRegistry()

# Set path of data directory

In [2]:
targets.set_data_dir(os.path.join(path))
# directory name where results for each target are stored
results_dir = '10_results'

# Number of targets, ligands and edges in the data set

In [3]:
nligs, nedgs = 0, 0
print(f'{"Target":10s} {"Num Ligs":>10s} {"Num Edges":>10s}')
print(33 * '-')
for target in tqdm(targets.target_dict):
    print(f'{target:10s} {len(ligands.LigandSet(target)):10d} {len(edges.EdgeSet(target)):10d}')
    nligs += len(ligands.LigandSet(target))
    nedgs += len(edges.EdgeSet(target))
print(33 * '-')
print(f'{"total":10s} {nligs:10d} {nedgs:10d}')

Target       Num Ligs  Num Edges
---------------------------------


  0%|          | 0/22 [00:00<?, ?it/s]

jnk1               21         31
pde2               21         34
thrombin           11         16
p38                34         56
ptp1b              23         49
galectin            8          7
cdk2               16         25
cmet               24         57
mcl1               42         71
bace               36         58
bace_hunt          32         60
bace_p2            12         26
tyk2               16         24
ros1               28         61
eg5                28         65
cdk8               33         54
hif2a              42         92
pfkfb3             40         66
pde10              35         59
shp2               26         56
syk                44        101
tnks2              27         60
---------------------------------
total             599       1128


# Gather data from Gapsys et al.

In [4]:
import subprocess
for forcefield in ['gaff2_sh', 'cgenff']:
    for target in [
               #'bace', 
               #'bace_hunt', 
               #'bace_p2', 
               #'cdk2', 
               #'galectin', 
               #'jnk1', 
 #              'mcl1', 
 #              'p38', 
               'pde10', 
 #              'pde2', 
 #              'ptp1b', 
               'ros1', 
 #              'thrombin', 
 #              'tyk2'
              ]:
        print(target)
        # workpath/[water|complex]/edge* - every edge has its own folder
        waterComplex = ['water','complex']
        # workpath/[water|complex]/edge*/state[A|B] - two states will be considered for every edge
        states = ['stateA','stateB']
        
        if forcefield == 'gaff2_sh':
            nFF = 'gaff2-sh'
        else:
            nFF = forcefield
            
        for edge in edges.EdgeSet(target):
            print(edge)
    #         pmxworkflow.printInfo(runtype='Analyse', run='', target=pwf.target, edge=edge, wc='', state='')
            for wc in waterComplex:
                print(f'    - {wc}')
                for run in [1, 2, 3]:
                    os.makedirs(f'analyze/{forcefield}/{target}/{wc}/{edge}/analyze{run}/', exist_ok=True)

                    # input files
    #                 fa = [f'{pwf.runPath}/{edge}/{wc}/stateA/morphes{run}/dgdl{i}.xvg' for i in range(1, 81)]
    #                 fb = [f'{pwf.runPath}/{edge}/{wc}/stateB/morphes{run}/dgdl{i}.xvg' for i in range(1, 81)]
    #               # output files
                    result = f'analyze/{forcefield}/{target}/{wc}/{edge}/analyze{run}/results.txt'
                    if wc == 'complex':
                        outWorkA = f'ros1_pde10/output-{nFF}-run3-protein/{target}/{edge}/stateA/run{run}/work.dat'
                        outWorkB = f'ros1_pde10/output-{nFF}-run3-protein/{target}/{edge}/stateB/run{run}/work.dat'
                    elif wc == 'water':
                        outWorkA = f'ros1_pde10/output-{nFF}-run3-water/{target}/{edge}/stateA/run{run}/work.dat'
                        outWorkB = f'ros1_pde10/output-{nFF}-run3-water/{target}/{edge}/stateB/run{run}/work.dat'
                    else:
                        raise Exception("wrong wc.")
                    # plot = f'{pwf.runPath}/{edge}/{wc}/analyse{run}/plot.png'

                    cmdline = f'pmx analyse '\
                    f'-o {result} '\
                    f'-iA {outWorkA} '\
                    f'-iB {outWorkB} '\
                    f'--work_plot none '\
                    f'-b 100 '\
                    f'-t 298 '\
                    f'-m bar '\
                    f'--no_ks '\
                    f'--unit kcal'

                    print(cmdline)
                    process = subprocess.run(cmdline.split(),
                                           stdout=subprocess.PIPE,
                                           stderr=subprocess.PIPE)
    #                 process.wait()


pde10
edge_7395_3484
    - water
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_7395_3484/analyze1/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_7395_3484/stateA/run1/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_7395_3484/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_7395_3484/analyze2/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_7395_3484/stateA/run2/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_7395_3484/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_7395_3484/analyze3/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_7395_3484/stateA/run3/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_7395_3484/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/gaff2_sh/pde

pmx analyse -o analyze/gaff2_sh/pde10/water/edge_1939_3032/analyze3/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_1939_3032/stateA/run3/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_1939_3032/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_1939_3032/analyze1/results.txt -iA ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1939_3032/stateA/run1/work.dat -iB ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1939_3032/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_1939_3032/analyze2/results.txt -iA ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1939_3032/stateA/run2/work.dat -iB ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1939_3032/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_1939_

pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_3806_0340/analyze2/results.txt -iA ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_3806_0340/stateA/run2/work.dat -iB ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_3806_0340/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_3806_0340/analyze3/results.txt -iA ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_3806_0340/stateA/run3/work.dat -iB ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_3806_0340/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
edge_3806_1423
    - water
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_3806_1423/analyze1/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_3806_1423/stateA/run1/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_3806_1423/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/water

edge_4147_7846
    - water
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_4147_7846/analyze1/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_4147_7846/stateA/run1/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_4147_7846/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_4147_7846/analyze2/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_4147_7846/stateA/run2/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_4147_7846/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/water/edge_4147_7846/analyze3/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_4147_7846/stateA/run3/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_4147_7846/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/gaff2_sh/pde10/com

pmx analyse -o analyze/gaff2_sh/pde10/water/edge_1038_0848/analyze3/results.txt -iA ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_1038_0848/stateA/run3/work.dat -iB ros1_pde10/output-gaff2-sh-run3-water/pde10/edge_1038_0848/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_1038_0848/analyze1/results.txt -iA ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1038_0848/stateA/run1/work.dat -iB ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1038_0848/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_1038_0848/analyze2/results.txt -iA ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1038_0848/stateA/run2/work.dat -iB ros1_pde10/output-gaff2-sh-run3-protein/pde10/edge_1038_0848/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/gaff2_sh/pde10/complex/edge_1038_

...

pmx analyse -o analyze/cgenff/ros1/water/edge_9022_9373/analyze3/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_9022_9373/stateA/run3/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_9022_9373/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/cgenff/ros1/complex/edge_9022_9373/analyze1/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_9022_9373/stateA/run1/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_9022_9373/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/complex/edge_9022_9373/analyze2/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_9022_9373/stateA/run2/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_9022_9373/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/complex/edge_9022_9373/analyze3/results.txt -iA 

pmx analyse -o analyze/cgenff/ros1/complex/edge_1859_2879/analyze3/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_1859_2879/stateA/run3/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_1859_2879/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
edge_2359_1495
    - water
pmx analyse -o analyze/cgenff/ros1/water/edge_2359_1495/analyze1/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_2359_1495/stateA/run1/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_2359_1495/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/water/edge_2359_1495/analyze2/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_2359_1495/stateA/run2/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_2359_1495/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/water/edge_2359_1495/analyze3/results.txt

pmx analyse -o analyze/cgenff/ros1/water/edge_2359_3040/analyze3/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_2359_3040/stateA/run3/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_2359_3040/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/cgenff/ros1/complex/edge_2359_3040/analyze1/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_2359_3040/stateA/run1/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_2359_3040/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/complex/edge_2359_3040/analyze2/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_2359_3040/stateA/run2/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_2359_3040/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/complex/edge_2359_3040/analyze3/results.txt -iA 

pmx analyse -o analyze/cgenff/ros1/complex/edge_8550_7454/analyze3/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_8550_7454/stateA/run3/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_8550_7454/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
edge_6691_9373
    - water
pmx analyse -o analyze/cgenff/ros1/water/edge_6691_9373/analyze1/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_6691_9373/stateA/run1/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_6691_9373/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/water/edge_6691_9373/analyze2/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_6691_9373/stateA/run2/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_6691_9373/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/water/edge_6691_9373/analyze3/results.txt

pmx analyse -o analyze/cgenff/ros1/water/edge_8602_5109/analyze3/results.txt -iA ros1_pde10/output-cgenff-run3-water/ros1/edge_8602_5109/stateA/run3/work.dat -iB ros1_pde10/output-cgenff-run3-water/ros1/edge_8602_5109/stateB/run3/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
    - complex
pmx analyse -o analyze/cgenff/ros1/complex/edge_8602_5109/analyze1/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_8602_5109/stateA/run1/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_8602_5109/stateB/run1/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/complex/edge_8602_5109/analyze2/results.txt -iA ros1_pde10/output-cgenff-run3-protein/ros1/edge_8602_5109/stateA/run2/work.dat -iB ros1_pde10/output-cgenff-run3-protein/ros1/edge_8602_5109/stateB/run2/work.dat --work_plot none -b 100 -t 298 -m bar --no_ks --unit kcal
pmx analyse -o analyze/cgenff/ros1/complex/edge_8602_5109/analyze3/results.txt -iA 

In [5]:
def read_neq_results( fname ):
    if not os.path.exists(fname):
        print('File does not exist')
        return []
    fp = open(fname,'r')
    lines = fp.readlines()
    fp.close()
    out = []
    for l in lines:
        l = l.rstrip()
        foo = l.split()
        if 'BAR: dG' in l:
            out.append(float(foo[-2]))
        elif 'BAR: Std Err (bootstrap)' in l:
            out.append(float(foo[-2]))
        elif 'BAR: Std Err (analytical)' in l:
            out.append(float(foo[-2]))
        elif 'BAR: Conv' in l:
            out.append(float(foo[-1]))
    return(out)


In [6]:
def output_textfile(edgs, df, fname):
    fp = open(fname, 'w')
    fp.write('#%6s  %6s  %6s\n' % ('1_edge', '2_ddg', '3_ddg_err'))
    fp.write('#all values in kJ/mol\n')

    for i, row in edgs.iterrows():
        edge = f'edge_{row[0]}_{row[1]}'
        val = df.loc[f'{edge}_ddg', 'val']
        err = df.loc[f'{edge}_ddg', 'err']

        fp.write('%10s  %4.2f  %4.2f\n' % (edge, val, err))

    fp.close()


In [7]:
def getGapsysResults(target, forcefield):  
#     file_name = f'{temp_directory}/protLig_benchmark/ddg_data/{target}.dat'
#     if not os.path.exists(file_name):
#         print(f'File {file_name} does not exist.')
#         return None
#     data = pd.read_csv(f'{temp_directory}/protLig_benchmark/ddg_data/{target}.dat', sep='\s+', header=None, comment='#',
#                    names=['edge', 'exp', 'gaff', 'dgaff', 'cgenff', 'dcgenff', 'cons', 'dcons', 'fep5', 'dfep5', 'fep1', 'dfep1'])
#     data['dexp']=pd.Series([0.0]*data.shape[0])
    edgs = edges.EdgeSet(target).get_dataframe()

    bootnum = 1000
    ######## read into a data frame #########
    df = pd.DataFrame()
    arrays = [['water', 'complex', 'ddg'] * 3,
              [int(i / 3) + 1 for i in range(9)],
              [''] * 9]
    arrays = [[] * 9] * 3
    tuples = list(zip(*arrays))

    index = pd.MultiIndex.from_tuples(tuples, names=['leg', 'repeat', ''])
    newdf = pd.DataFrame(columns=index)

    # workpath/[water|complex]/edge* - every edge has its own folder
    waterComplex = ['water','complex']
    waterComplexNew = ['water','complex']
    
    if forcefield == 'openff-2.0.0-rc.1.offxml':
        nFF = 'openff-2.0rc1'
    elif forcefield == 'gaff':
        nFF = 'gaff2_sh'
    else:
        nFF = forcefield
    
    if not os.path.exists(f'analyze/{nFF}/{target}/'):
        return None
    # workpath/[water|complex]/edge*/state[A|B] - two states will be considered for every edge
    states = ['stateA','stateB']
    for i, row in edgs.iterrows():
        edge = f'edge_{row[0]}_{row[1]}'
        for wc, nwc in zip(waterComplex, waterComplexNew):
            for run in range(1,4):
                result = f'analyze/{nFF}/{target}/{wc}/{edge}/analyze{run}/results.txt'
                print(result)
                foo = read_neq_results(result)
                if len(foo) > 1:
                    df.loc[f'{edge}_{nwc}{run}','val'] = foo[0] #unit_registry.Quantity(foo[0], 'kilojoules/mole').to('kilocalories/mole').magnitude
                    df.loc[f'{edge}_{nwc}{run}','err'] = foo[2] #unit_registry.Quantity(foo[2], 'kilojoules/mole').to('kilocalories/mole').magnitude
                    df.loc[f'{edge}_{nwc}{run}','aerr'] = foo[1] #unit_registry.Quantity(foo[1], 'kilojoules/mole').to('kilocalories/mole').magnitude
                    df.loc[f'{edge}_{nwc}{run}','conv'] = foo[3]
                else:
                    df.loc[f'{edge}_{nwc}{run}','val'] = np.nan
                    df.loc[f'{edge}_{nwc}{run}','err'] = np.nan
                    df.loc[f'{edge}_{nwc}{run}','aerr'] = np.nan
                    df.loc[f'{edge}_{nwc}{run}','conv'] = np.nan
                    print('Results could not be read')
                for t in ['val', 'err', 'aerr', 'conv']:
                    newdf.loc[f'{edge}', (nwc, run, t)] = df.loc[f'{edge}_{nwc}{run}', t]
        vals = []
        errs = []
        aerrs = []
        for run in range(1,4):
            ##### calculate ddg #####
            ddg = df.loc[f'{edge}_complex{run}','val'] - df.loc[f'{edge}_water{run}','val']
            err = np.sqrt( np.power(df.loc[f'{edge}_complex{run}','err'],2.0) +
                           np.power(df.loc[f'{edge}_water{run}','err'],2.0) )
            aerr = np.sqrt( np.power(df.loc[f'{edge}_complex{run}','aerr'],2.0) +
                            np.power(df.loc[f'{edge}_water{run}','aerr'],2.0) )
            df.loc[f'{edge}_ddg{run}','val'] = ddg
            df.loc[f'{edge}_ddg{run}','err'] = err
            df.loc[f'{edge}_ddg{run}','aerr'] = aerr
            newdf.loc[f'{edge}', ('ddg', run, 'val')] = ddg
            newdf.loc[f'{edge}', ('ddg', run, 'err')] = err
            newdf.loc[f'{edge}', ('ddg', run, 'aerr')] = aerr
            vals.append(ddg)
            errs.append(err)
            aerrs.append(aerr)

        ###### calculate mean dg with err ######
        # mean ddg
        mean = np.average(vals)
        df.loc[f'{edge}_ddg', 'val'] = mean

        # error
        # 1) create three distributions
        distribs = []
        for v, e in zip(vals, errs):
            distribs.append(np.random.normal(v, e, size=bootnum))
        if len(distribs) > 1:
            distr = np.vstack(distribs)
            # 2) calculate stderrs
            stderr = np.mean(np.sqrt(np.var(distr, ddof=1, axis=0) / np.float(len(distribs))))
            df.loc[f'{edge}_ddg', 'err'] = stderr
        else:
            stderr = errs[0]
            df.loc[f'{edge}_ddg','err'] = stderr

        for t in ['val', 'err', 'aerr']:
            newdf.loc[f'{edge}', ('ddg_mean', '-', t)] = df.loc[f'{edge}_ddg', t]
    ###### output ######
    output_textfile(edgs, df, os.path.join(path, '..', '00_data', 'input', f'{target}_{forcefield}.dat'))

    newdf.to_csv(os.path.join(path, '..', '00_data', 'input', f'{target}_{forcefield}.csv'), float_format='%10.2f')


#     newdata = data.copy()
#     newdata.index=newdata['edge']
#     newdata.columns = pd.MultiIndex.from_arrays([np.array(newdata.columns), ['', 'exp'] + ['pmx'] * 6 + ['fep'] * 4 + ['exp'], [''] + ['kj/mol'] * 12], names=['forcefield', 'method', 'unit'])
    
#     newdata.loc[:,('ligandA', '', '')] = df[0].apply(lambda x: str(int(float(x))) if str(x).endswith('.0') else x).astype(str)
#     newdata.loc[:,('ligandB', '', '')] = df[1].apply(lambda x: str(int(float(x))) if str(x).endswith('.0') else x).astype(str)
#     newdata = newdata.drop(columns=('edge', '', ''))
#     newdata.sort_index(axis=1, level=1, inplace=True, sort_remaining=False)
#     newdata.index=[f'{target}_edge_{x[("ligandA", "", "")]}_{x[("ligandB", "", "")]}' for i, x in newdata.iterrows()]
    
#     return newdata
# getGapsysResults('cmet')

In [8]:
# function to retrieve data from PLBenchmarks calculations, soted in 00_data/input 
def getRawResults(target, forcefield='openff-1.0.0.offxml'):
    file_path = os.path.join(path, 
                                   '..', 
                                   '00_data', 
                                   'input',  
                                   f'{target}_{forcefield}.dat'
                                  )
    if not os.path.exists(file_path):
        print(f'File {file_path} does not exist.')
        return
    # read in result file
    res = pd.read_csv(file_path,
                      header=None, 
                      comment='#', 
                      skipinitialspace=True, 
                      names=['edge', 'calc DDG', 'calc dDDG' , 'add dDDG'], 
                      sep=' ')
    res.index = res['edge']
    # read in exp. data
    edg = edges.EdgeSet(target)
    df = edg.get_dataframe(columns=[0,1, 'exp. DeltaG [kcal/mol]', 'exp. Error [kcal/mol]'])
    df.index = pd.Series(['edge_' + str(lig1) + '_' + str(lig2) for lig1, lig2 in zip(df[0].values, df[1].values)])
   
    # copy in calculated values
    df['calc_DDG'] = res.loc[:,'calc DDG']
    df['calc_dDDG'] = res.loc[:,'calc dDDG']
    df['calc_dDDG(additional)'] = res.loc[:, 'add dDDG']
    
    # remove unit of calculated values
    df['exp_DDG'] = df['exp. DeltaG [kcal/mol]'].apply(lambda x: x.magnitude)
    df['exp_dDDG'] = df['exp. Error [kcal/mol]'].apply(lambda x: x.magnitude)

    # filter only to relevant columns
    df = df.filter(items = [0,1,'exp_DDG', 'exp_dDDG', 'calc_DDG', 'calc_dDDG', 'calc_dDDG(additional)'])
    df[0]= df[0].astype(str)
    df[1]= df[1].astype(str)
    df.index = pd.Series([f'{target}_edge_' + str(lig1) + '_' + str(lig2) for lig1, lig2 in zip(df[0].values, df[1].values)])
    return df

In [9]:
author = "gapsys"
software = "pmx"
for target in ['pde10', 'ros1']:
    print(target)
    for forcefield in ['gaff', 'cgenff']:
        print(f"    {forcefield}")
        
        getGapsysResults(target, forcefield)
        df = getRawResults(target, forcefield)
        if df is None:
            continue
        os.makedirs(os.path.join(path, targets.get_target_dir(target), results_dir), exist_ok=True)
        with open(os.path.join(path, targets.get_target_dir(target), results_dir, f'{target}_{software}_{forcefield}_{author}.yaml'), 'w') as file:
            df = df.filter([0,1,'calc_DDG', 'calc_dDDG'])
            df.rename(columns={0: 'ligandA', 1: 'ligandB', 'calc_DDG': 'DDG', 'calc_dDDG': 'dDDG'}, inplace=True)
            df['unit']='kilocalories / mole'
            yaml.dump(df.T.to_dict(), file)

pde10
    gaff
analyze/gaff2_sh/pde10/water/edge_7395_3484/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_7395_3484/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_7395_3484/analyze3/results.txt
analyze/gaff2_sh/pde10/complex/edge_7395_3484/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_7395_3484/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_7395_3484/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_7395_0738/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_7395_0738/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_7395_0738/analyze3/results.txt
analyze/gaff2_sh/pde10/complex/edge_7395_0738/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_7395_0738/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_7395_0738/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_8041_4415/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_8041_4415/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_8041_4415/analyze3/results.tx

analyze/gaff2_sh/pde10/complex/edge_0395_8161/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_0395_8161/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_0395_8161/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_8414_1038/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_8414_1038/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_8414_1038/analyze3/results.txt
analyze/gaff2_sh/pde10/complex/edge_8414_1038/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_8414_1038/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_8414_1038/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_8414_1056/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_8414_1056/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_8414_1056/analyze3/results.txt
analyze/gaff2_sh/pde10/complex/edge_8414_1056/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_8414_1056/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_8414_1056/analyze3/results.txt
analyze

analyze/gaff2_sh/pde10/complex/edge_1038_5687/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_4415_3032/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_4415_3032/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_4415_3032/analyze3/results.txt
analyze/gaff2_sh/pde10/complex/edge_4415_3032/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_4415_3032/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_4415_3032/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_3806_4147/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_3806_4147/analyze2/results.txt
analyze/gaff2_sh/pde10/water/edge_3806_4147/analyze3/results.txt
analyze/gaff2_sh/pde10/complex/edge_3806_4147/analyze1/results.txt
analyze/gaff2_sh/pde10/complex/edge_3806_4147/analyze2/results.txt
analyze/gaff2_sh/pde10/complex/edge_3806_4147/analyze3/results.txt
analyze/gaff2_sh/pde10/water/edge_1423_4754/analyze1/results.txt
analyze/gaff2_sh/pde10/water/edge_1423_4754/analyze2/results.txt
analyze/gaf

analyze/cgenff/pde10/water/edge_4147_2750/analyze1/results.txt
analyze/cgenff/pde10/water/edge_4147_2750/analyze2/results.txt
analyze/cgenff/pde10/water/edge_4147_2750/analyze3/results.txt
analyze/cgenff/pde10/complex/edge_4147_2750/analyze1/results.txt
analyze/cgenff/pde10/complex/edge_4147_2750/analyze2/results.txt
analyze/cgenff/pde10/complex/edge_4147_2750/analyze3/results.txt
analyze/cgenff/pde10/water/edge_4147_7846/analyze1/results.txt
analyze/cgenff/pde10/water/edge_4147_7846/analyze2/results.txt
analyze/cgenff/pde10/water/edge_4147_7846/analyze3/results.txt
analyze/cgenff/pde10/complex/edge_4147_7846/analyze1/results.txt
analyze/cgenff/pde10/complex/edge_4147_7846/analyze2/results.txt
analyze/cgenff/pde10/complex/edge_4147_7846/analyze3/results.txt
analyze/cgenff/pde10/water/edge_6694_4147/analyze1/results.txt
analyze/cgenff/pde10/water/edge_6694_4147/analyze2/results.txt
analyze/cgenff/pde10/water/edge_6694_4147/analyze3/results.txt
analyze/cgenff/pde10/complex/edge_6694_4147

analyze/cgenff/pde10/complex/edge_7395_1939/analyze2/results.txt
analyze/cgenff/pde10/complex/edge_7395_1939/analyze3/results.txt
analyze/cgenff/pde10/water/edge_5687_8161/analyze1/results.txt
analyze/cgenff/pde10/water/edge_5687_8161/analyze2/results.txt
analyze/cgenff/pde10/water/edge_5687_8161/analyze3/results.txt
analyze/cgenff/pde10/complex/edge_5687_8161/analyze1/results.txt
analyze/cgenff/pde10/complex/edge_5687_8161/analyze2/results.txt
analyze/cgenff/pde10/complex/edge_5687_8161/analyze3/results.txt
analyze/cgenff/pde10/water/edge_5544_4147/analyze1/results.txt
analyze/cgenff/pde10/water/edge_5544_4147/analyze2/results.txt
analyze/cgenff/pde10/water/edge_5544_4147/analyze3/results.txt
analyze/cgenff/pde10/complex/edge_5544_4147/analyze1/results.txt
analyze/cgenff/pde10/complex/edge_5544_4147/analyze2/results.txt
analyze/cgenff/pde10/complex/edge_5544_4147/analyze3/results.txt
analyze/cgenff/pde10/water/edge_8041_0738/analyze1/results.txt
analyze/cgenff/pde10/water/edge_8041_07

analyze/gaff2_sh/ros1/complex/edge_3053_8602/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_3053_8602/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_3053_8602/analyze3/results.txt
analyze/gaff2_sh/ros1/water/edge_0529_6770/analyze1/results.txt
analyze/gaff2_sh/ros1/water/edge_0529_6770/analyze2/results.txt
analyze/gaff2_sh/ros1/water/edge_0529_6770/analyze3/results.txt
analyze/gaff2_sh/ros1/complex/edge_0529_6770/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_0529_6770/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_0529_6770/analyze3/results.txt
analyze/gaff2_sh/ros1/water/edge_0681_6770/analyze1/results.txt
analyze/gaff2_sh/ros1/water/edge_0681_6770/analyze2/results.txt
analyze/gaff2_sh/ros1/water/edge_0681_6770/analyze3/results.txt
analyze/gaff2_sh/ros1/complex/edge_0681_6770/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_0681_6770/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_0681_6770/analyze3/results.txt
analyze/gaff2_sh/ros1/

analyze/gaff2_sh/ros1/complex/edge_9373_5602/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_9373_5602/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_9373_5602/analyze3/results.txt
analyze/gaff2_sh/ros1/water/edge_9373_0529/analyze1/results.txt
analyze/gaff2_sh/ros1/water/edge_9373_0529/analyze2/results.txt
analyze/gaff2_sh/ros1/water/edge_9373_0529/analyze3/results.txt
analyze/gaff2_sh/ros1/complex/edge_9373_0529/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_9373_0529/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_9373_0529/analyze3/results.txt
analyze/gaff2_sh/ros1/water/edge_8794_6674/analyze1/results.txt
analyze/gaff2_sh/ros1/water/edge_8794_6674/analyze2/results.txt
analyze/gaff2_sh/ros1/water/edge_8794_6674/analyze3/results.txt
analyze/gaff2_sh/ros1/complex/edge_8794_6674/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_8794_6674/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_8794_6674/analyze3/results.txt
analyze/gaff2_sh/ros1/

analyze/gaff2_sh/ros1/complex/edge_8550_7454/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_8550_7454/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_8550_7454/analyze3/results.txt
analyze/gaff2_sh/ros1/water/edge_6691_9373/analyze1/results.txt
analyze/gaff2_sh/ros1/water/edge_6691_9373/analyze2/results.txt
analyze/gaff2_sh/ros1/water/edge_6691_9373/analyze3/results.txt
analyze/gaff2_sh/ros1/complex/edge_6691_9373/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_6691_9373/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_6691_9373/analyze3/results.txt
analyze/gaff2_sh/ros1/water/edge_8602_2606/analyze1/results.txt
analyze/gaff2_sh/ros1/water/edge_8602_2606/analyze2/results.txt
analyze/gaff2_sh/ros1/water/edge_8602_2606/analyze3/results.txt
analyze/gaff2_sh/ros1/complex/edge_8602_2606/analyze1/results.txt
analyze/gaff2_sh/ros1/complex/edge_8602_2606/analyze2/results.txt
analyze/gaff2_sh/ros1/complex/edge_8602_2606/analyze3/results.txt
analyze/gaff2_sh/ros1/

analyze/cgenff/ros1/water/edge_1537_6770/analyze1/results.txt
analyze/cgenff/ros1/water/edge_1537_6770/analyze2/results.txt
analyze/cgenff/ros1/water/edge_1537_6770/analyze3/results.txt
analyze/cgenff/ros1/complex/edge_1537_6770/analyze1/results.txt
analyze/cgenff/ros1/complex/edge_1537_6770/analyze2/results.txt
analyze/cgenff/ros1/complex/edge_1537_6770/analyze3/results.txt
analyze/cgenff/ros1/water/edge_1859_9373/analyze1/results.txt
analyze/cgenff/ros1/water/edge_1859_9373/analyze2/results.txt
analyze/cgenff/ros1/water/edge_1859_9373/analyze3/results.txt
analyze/cgenff/ros1/complex/edge_1859_9373/analyze1/results.txt
analyze/cgenff/ros1/complex/edge_1859_9373/analyze2/results.txt
analyze/cgenff/ros1/complex/edge_1859_9373/analyze3/results.txt
analyze/cgenff/ros1/water/edge_1859_8654/analyze1/results.txt
analyze/cgenff/ros1/water/edge_1859_8654/analyze2/results.txt
analyze/cgenff/ros1/water/edge_1859_8654/analyze3/results.txt
analyze/cgenff/ros1/complex/edge_1859_8654/analyze1/result

analyze/cgenff/ros1/water/edge_9022_2879/analyze2/results.txt
analyze/cgenff/ros1/water/edge_9022_2879/analyze3/results.txt
analyze/cgenff/ros1/complex/edge_9022_2879/analyze1/results.txt
analyze/cgenff/ros1/complex/edge_9022_2879/analyze2/results.txt
analyze/cgenff/ros1/complex/edge_9022_2879/analyze3/results.txt
analyze/cgenff/ros1/water/edge_7447_2606/analyze1/results.txt
analyze/cgenff/ros1/water/edge_7447_2606/analyze2/results.txt
analyze/cgenff/ros1/water/edge_7447_2606/analyze3/results.txt
analyze/cgenff/ros1/complex/edge_7447_2606/analyze1/results.txt
analyze/cgenff/ros1/complex/edge_7447_2606/analyze2/results.txt
analyze/cgenff/ros1/complex/edge_7447_2606/analyze3/results.txt
analyze/cgenff/ros1/water/edge_7447_4451/analyze1/results.txt
analyze/cgenff/ros1/water/edge_7447_4451/analyze2/results.txt
analyze/cgenff/ros1/water/edge_7447_4451/analyze3/results.txt
analyze/cgenff/ros1/complex/edge_7447_4451/analyze1/results.txt
analyze/cgenff/ros1/complex/edge_7447_4451/analyze2/resu